## Nettoyage des données

In [1037]:
import pandas as pd
import re

In [1038]:
data_frame = pd.read_csv('export_alimconfiance@dgal.csv', sep=';')

In [1039]:
data_frame

,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite
0,SARL AUBERGE DU DOUBLE SIX,30121048000010,24 PL GENERAL LECLERC,80220,GAMACHES,16793484,2023-04-25T02:00:00+02:00,Restaurant,Satisfaisant,NaN,"49.985274, 1.561386",Restaurant,Autres
1,MONSIEUR PHILIPPE MARIOTTE,31084873400029,286 RUE DU CENTRE,74260,LES GETS,16740394,2023-02-23T01:00:00+01:00,Restaurant,Satisfaisant,NaN,"46.15861, 6.677963",Restaurant,Autres
2,NATURALIA (NATURALIA FRANCE),30247464801894,36 RUE EUGENE EICHENBERGER,92800,PUTEAUX,16611978,2022-10-04T02:00:00+02:00,Alimentation générale,Satisfaisant,NaN,"48.881317, 2.23746",Alimentation générale,Autres
3,SUPER-U (COLOMBELLES DISTRIBUTION),30122377200031,RD 403,14460,COLOMBELLES,16787460,2023-04-20T02:00:00+02:00,Rayon boucherie-charcuterie,Très satisfaisant,NaN,NaN,Rayon boucherie-charcuterie,Autres
4,FRANPRIX (SOC DISTRIBUTION SODISCO SARL),33427109500013,"4, PLACE DU MOUSTIER",92210,SAINT CLOUD,16748988,2023-03-08T01:00:00+01:00,Alimentation générale,A améliorer,NaN,"48.844091, 2.219818",Alimentation générale,Autres
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32715,EARL TOUT LE BON,90449950600015,LE PLATE,97424,Saint-Leu,23-012756-1,2023-03-30T02:00:00+02:00,Chaîne d'abattage de volaille/lagomorphe/petit...,Satisfaisant,NaN,"-21.206368, 55.337063",NaN,Abattoirs
32716,LA POKETERIE,84475120600013,3 RUE DE PROVENCE,75009,Paris 9e Arrondissement,22-094901-1,2022-11-24T01:00:00+01:00,Restaurant,Satisfaisant,NaN,"48.873997, 2.341843",Restaurant,Autres
32717,SAS GUY COQUILLAT,38853882900010,Z A Les Blaches Gombert,04160,Château-Arnoux-Saint-Auban,22-092105-1,2023-04-17T02:00:00+02:00,Transformation de produits carnés,Satisfaisant,04049001,"44.078462, 5.999322",NaN,Viandes et produits carnés
32718,LIVESTATION DIY,75169891100011,LIVESTATION DIY,69007,Lyon 7e Arrondissement,23-047240-1,2023-06-16T02:00:00+02:00,Restaurant,Satisfaisant,NaN,"45.738402, 4.837636",Restaurant,Autres


In [1040]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32720 entries, 0 to 32719
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   APP_Libelle_etablissement           32719 non-null  object
 1   SIRET                               32720 non-null  object
 2   Adresse_2_UA                        32292 non-null  object
 3   Code_postal                         32720 non-null  object
 4   Libelle_commune                     32720 non-null  object
 5   Numero_inspection                   32720 non-null  object
 6   Date_inspection                     32720 non-null  object
 7   APP_Libelle_activite_etablissement  32720 non-null  object
 8   Synthese_eval_sanit                 32720 non-null  object
 9   Agrement                            8787 non-null   object
 10  geores                              31961 non-null  object
 11  filtre                              24086 non-null  ob

In [1041]:
data_frame.describe()

,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite
count,32719,32720,32292,32720,32720,32720,32720,32720,32720,8787,31961,24086,32720
unique,25744,28832,27101,4989,8460,32720,299,149,4,6835,26914,94,7
top,ECOLE PRIMAIRE PUBLIQUE,39814347900069,LE BOURG,75015,Paris 15e Arrondissement,16793484,2023-04-04T02:00:00+02:00,Restaurant,Satisfaisant,72080002,"48.859, 2.347",Restaurant,Autres
freq,162,12,91,557,536,1,251,11653,17976,10,50,11653,24737


In [1042]:
data_frame.columns

Index(['APP_Libelle_etablissement', 'SIRET', 'Adresse_2_UA', 'Code_postal',
       'Libelle_commune', 'Numero_inspection', 'Date_inspection',
       'APP_Libelle_activite_etablissement', 'Synthese_eval_sanit', 'Agrement',
       'geores', 'filtre', 'ods_type_activite'],
      dtype='object')

In [1043]:
# Droper les colonnes qui ne sont pas :  Date_inspection, APP_Libelle_activite, geores, Synthese_eval_sanitaire
data_frame = data_frame.drop(['APP_Libelle_etablissement', 'SIRET', 'Adresse_2_UA', 'Code_postal', 'Agrement', 'filtre', 'ods_type_activite', 'Numero_inspection', 'Libelle_commune'], axis=1)

In [1044]:
data_frame.columns

Index(['Date_inspection', 'APP_Libelle_activite_etablissement',
       'Synthese_eval_sanit', 'geores'],
      dtype='object')

In [1045]:
# On decompose les datas en 2 parties : les features et les targets
# Les features sont les colonnes qui nous permettent de predire la target
# Les targets sont les colonnes est la colonne Synthese_eval_sanit

X = data_frame.drop(['Synthese_eval_sanit'], axis=1)
Y = data_frame['Synthese_eval_sanit']

In [1046]:
X

,Date_inspection,APP_Libelle_activite_etablissement,geores
0,2023-04-25T02:00:00+02:00,Restaurant,"49.985274, 1.561386"
1,2023-02-23T01:00:00+01:00,Restaurant,"46.15861, 6.677963"
2,2022-10-04T02:00:00+02:00,Alimentation générale,"48.881317, 2.23746"
3,2023-04-20T02:00:00+02:00,Rayon boucherie-charcuterie,NaN
4,2023-03-08T01:00:00+01:00,Alimentation générale,"48.844091, 2.219818"
...,...,...,...
32715,2023-03-30T02:00:00+02:00,Chaîne d'abattage de volaille/lagomorphe/petit...,"-21.206368, 55.337063"
32716,2022-11-24T01:00:00+01:00,Restaurant,"48.873997, 2.341843"
32717,2023-04-17T02:00:00+02:00,Transformation de produits carnés,"44.078462, 5.999322"
32718,2023-06-16T02:00:00+02:00,Restaurant,"45.738402, 4.837636"


In [1047]:
Y

0             Satisfaisant
1             Satisfaisant
2             Satisfaisant
3        Très satisfaisant
4              A améliorer
               ...        
32715         Satisfaisant
32716         Satisfaisant
32717         Satisfaisant
32718         Satisfaisant
32719         Satisfaisant
Name: Synthese_eval_sanit, Length: 32720, dtype: object

In [1048]:
# On crée une fonction qui va nous permettre de vérifier le format de la date avec cet example 2023-04-25T02:00:00+02:00, si la ligne est pas bonne on drop la ligne sinon on retourne la date en timestamp
def check_date_format(date):
    if re.match(r'^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\+\d{2}:\d{2}$', date):
        return date
    else:
        return None

In [1049]:
# On applique la fonction sur la colonne Date_inspection
X['Date_inspection'] = X['Date_inspection'].apply(check_date_format)

# Transforme ' - ' value en null sur la colonne APP_Libelle_activite_etablissement
X['APP_Libelle_activite_etablissement'] = X['APP_Libelle_activite_etablissement'].replace('_', None)

# On drop les lignes qui ont une valeur null ou NaN
X = X.dropna()

# split colonne geores en 2 colonnes : latitude et longitude
X[['latitude', 'longitude']] = X['geores'].str.split(',', expand=True)
X = X.drop(['geores'], axis=1)

/Users/quentin/opt/anaconda3/envs/epsi/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [1050]:
X

,Date_inspection,APP_Libelle_activite_etablissement,latitude,longitude
0,2023-04-25T02:00:00+02:00,Restaurant,49.985274,1.561386
1,2023-02-23T01:00:00+01:00,Restaurant,46.15861,6.677963
2,2022-10-04T02:00:00+02:00,Alimentation générale,48.881317,2.23746
4,2023-03-08T01:00:00+01:00,Alimentation générale,48.844091,2.219818
5,2023-04-25T02:00:00+02:00,Alimentation générale,48.072687,-4.185142
...,...,...,...,...
32715,2023-03-30T02:00:00+02:00,Chaîne d'abattage de volaille/lagomorphe/petit...,-21.206368,55.337063
32716,2022-11-24T01:00:00+01:00,Restaurant,48.873997,2.341843
32717,2023-04-17T02:00:00+02:00,Transformation de produits carnés,44.078462,5.999322
32718,2023-06-16T02:00:00+02:00,Restaurant,45.738402,4.837636


In [1051]:
# On créer une fonction qui supprime les caractères spéciaux et les espaces dans une chaine de caractère
def remove_special_characters_and_spaces(string):
    return re.sub(r'[^a-zA-Z0-9]', '', string)

In [1052]:
# On applique la fonction sur toutes les colonnes
X['APP_Libelle_activite_etablissement'] = X['APP_Libelle_activite_etablissement'].apply(remove_special_characters_and_spaces)
X['latitude'] = X['latitude'].apply(remove_special_characters_and_spaces)
X['longitude'] = X['longitude'].apply(remove_special_characters_and_spaces)
X['Date_inspection'] = X['Date_inspection'].apply(remove_special_characters_and_spaces)

In [1053]:
# On split la colonne Date_inspection en 2 colonnes : date et heure
X[['date', 'heure']] = X['Date_inspection'].str.split('T', expand=True)

# On drop la colonne Date_inspection
X = X.drop(['Date_inspection'], axis=1)

In [1054]:
X

,APP_Libelle_activite_etablissement,latitude,longitude,date,heure
0,Restaurant,49985274,1561386,20230425,0200000200
1,Restaurant,4615861,6677963,20230223,0100000100
2,Alimentationgnrale,48881317,223746,20221004,0200000200
4,Alimentationgnrale,48844091,2219818,20230308,0100000100
5,Alimentationgnrale,48072687,4185142,20230425,0200000200
...,...,...,...,...,...
32715,Chanedabattagedevolaillelagomorphepetitgibier,21206368,55337063,20230330,0200000200
32716,Restaurant,48873997,2341843,20221124,0100000100
32717,Transformationdeproduitscarns,44078462,5999322,20230417,0200000200
32718,Restaurant,45738402,4837636,20230616,0200000200


In [1055]:
# On transforme les colonnes latitude et longitude en float
X['latitude'] = X['latitude'].astype(float)
X['longitude'] = X['longitude'].astype(float)

In [1056]:
X

,APP_Libelle_activite_etablissement,latitude,longitude,date,heure
0,Restaurant,49985274.0,1561386.0,20230425,0200000200
1,Restaurant,4615861.0,6677963.0,20230223,0100000100
2,Alimentationgnrale,48881317.0,223746.0,20221004,0200000200
4,Alimentationgnrale,48844091.0,2219818.0,20230308,0100000100
5,Alimentationgnrale,48072687.0,4185142.0,20230425,0200000200
...,...,...,...,...,...
32715,Chanedabattagedevolaillelagomorphepetitgibier,21206368.0,55337063.0,20230330,0200000200
32716,Restaurant,48873997.0,2341843.0,20221124,0100000100
32717,Transformationdeproduitscarns,44078462.0,5999322.0,20230417,0200000200
32718,Restaurant,45738402.0,4837636.0,20230616,0200000200


In [1057]:
# Formate les données avec le get_dummies
X = pd.get_dummies(X)

In [1058]:
X

,latitude,longitude,APP_Libelle_activite_etablissement_Abattagedeproduitsdelaquaculture,APP_Libelle_activite_etablissement_Alimentationgnrale,APP_Libelle_activite_etablissement_AlimentationgnraleBoucherieCharcuterie,APP_Libelle_activite_etablissement_AlimentationgnraleBoulangeriePtisserie,APP_Libelle_activite_etablissement_AlimentationgnraleBoulangeriePtisserieGlacierPoissonnerieBoucherieCharcuterie,APP_Libelle_activite_etablissement_AlimentationgnraleBoulangeriePtisserieTraiteur,APP_Libelle_activite_etablissement_AlimentationgnraleGlacier,APP_Libelle_activite_etablissement_AlimentationgnralePoissonnerie,...,date_20230831,date_20230901,date_20230902,date_20230904,date_20230905,date_20230906,date_20230907,date_20230908,heure_0100000100,heure_0200000200
0,49985274.0,1561386.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,4615861.0,6677963.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,48881317.0,223746.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,48844091.0,2219818.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,48072687.0,4185142.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32715,21206368.0,55337063.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
32716,48873997.0,2341843.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
32717,44078462.0,5999322.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
32718,45738402.0,4837636.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [1059]:
# On crée une fonction qui va nous permettre de transformer les valeurs de la colonne Synthese_eval_sanit en 0 ou 1
def transform_synthese_eval_sanit(value):
    if value == 'Très satisfaisant':
        return 1
    else:
        return 0

In [1060]:
# On applique la fonction sur la colonne Synthese_eval_sanit
Y = Y.apply(transform_synthese_eval_sanit)

In [1061]:
Y

0        0
1        0
2        0
3        1
4        0
        ..
32715    0
32716    0
32717    0
32718    0
32719    0
Name: Synthese_eval_sanit, Length: 32720, dtype: int64